In [1]:
import numpy as np
import copy as cp
from tkinter import *
import time
import os

ligneMax = 10
colonneMax = 10
delta = 0
gamma = 0.9

plateau = np.zeros((ligneMax,colonneMax))
start = (0, 0)
goal = (9, 9)
walls = [(0, 1),(1, 1),(0, 3),(0, 5),(0, 6),(1, 3),(1, 8),(2, 5),(3, 0),(3, 1),(3, 3),(3, 4),(3, 5),(3, 7),(4, 5),(4, 8),(4, 9),(5, 1),(5, 3),(5, 6),(5, 8),(6, 1),(6, 4),(6, 6),(6, 7),(6, 8),(7, 2),(7, 5),(7, 6),(8, 0),(8, 3),(8, 4),(8, 9),(9, 1),(9, 6),(8, 8)]

# Largeur du canevas
lg=30*colonneMax
# Hauteur du canevas
ht=30*ligneMax
# Largeur du bord
bw=50
# Largeur de la zone de focus
zf=10
 
fen1 = Tk()
fen1.title("Labirynthe un peu crazy")
 
# creation d'un widget 'canvas' contenant des images bitmap:
 
can1=Canvas(fen1, width =lg, height = ht, bd = 0, bg ='white')

logoWall = PhotoImage(file="../images/wall.png")
logoStart = PhotoImage(file="../images/start.png")
logoFinish = PhotoImage(file="../images/arrivee.png")
logoPerso = PhotoImage(file="../images/perso.png")

ystart, xstart = start
yfinish, xfinish = goal
itemstart = can1.create_image(((xstart*30)+18),((ystart*30)+18),image=logoStart)
itemfinish = can1.create_image(((xfinish*30)+18),((yfinish*30)+18),image=logoFinish)
for w in walls:
    wy, wx = w
    item = can1.create_image(((wx*30)+18),((wy*30)+18),image=logoWall)    
itemperso = can1.create_image(((xstart*30)+18),((ystart*30)+18),image=logoPerso)
can1.grid()

# Set of states
states = [(ligne, colonne) for ligne in range(ligneMax-1) for colonne in range(colonneMax-1)]

# Set of actions
actions = ["left", "right", "up", "down"]

# Transition function
def transition(state, action):
    i, j = state
    if action == "left":
        return (i, max(0, j-1))
    elif action == "right":
        return (i, min(colonneMax-1, j+1))
    elif action == "up":
        return (max(0, i-1), j)
    elif action == "down":
        return (min(ligneMax-1, i+1), j)

# Reward function
def reward(state):
    if state == goal:
        return 10
    elif state in walls:
        return -100
    else:
        return -1
    
#Parcours le plateau
def parcoursPlateau(plateau):
    newPlateau = cp.copy(plateau)
    listRewards=[]
    for l in range (0, ligneMax):
        for c in range (0, colonneMax):
            listRewards.clear()
            state = (l,c)
            reward_actuel = reward(state)
            
            #Reward qu'on aurait en allant a gauche
            if (c-1 >= 0):
                i_temp,j_temp = transition(state,"left")
                temp_reward = reward_actuel + gamma * plateau[i_temp][j_temp]
                listRewards.append(temp_reward)
                
            #Reward qu'on aurait en allant a droite
            if (c+1 < colonneMax):
                i_temp,j_temp = transition(state,"right")
                temp_reward = reward_actuel + gamma * plateau[i_temp][j_temp]
                listRewards.append(temp_reward)
                
            #Reward qu'on aurait en allant en bas
            if (l-1 >= 0):
                i_temp,j_temp = transition(state,"up")
                temp_reward = reward_actuel + gamma * plateau[i_temp][j_temp]
                listRewards.append(temp_reward)
                
            #Reward qu'on aurait en allant en haut
            if (l+1 < ligneMax):
                i_temp,j_temp = transition(state,"down")
                temp_reward = reward_actuel + gamma * plateau[i_temp][j_temp]
                listRewards.append(temp_reward)
                
            #Trouve le meilleur reward
            finalReward = round(max(listRewards),0)
            
            #Actualisation plateau
            newPlateau[state]=finalReward           
    
    return newPlateau
            
        
def plateauDifferents(old,new,erreur):
    for l in range (0, ligneMax):
        for c in range (0, colonneMax):
            if ((new[l,c] > old[l,c]+erreur)or(new[l,c]<old[l,c]-erreur)):
                return True
    return False

def trouverModele(plateau, iterationsMax):
    oldPlateau = np.zeros((ligneMax,colonneMax))
    oldPlateau[ligneMax-2, colonneMax-1] = 2
    nbIteration = 0
    while (plateauDifferents(oldPlateau,plateau,delta) and (nbIteration < iterationsMax)):
        oldPlateau = cp.copy(plateau)
        plateau = parcoursPlateau(oldPlateau)
        nbIteration = nbIteration + 1
    print("Le modèle à été trouvé en", nbIteration, "itérations")
    return plateau

def deplacerPerso(state,plateau):
    y, x = state
    listPossibilite=[]
    
    if (x-1 >= 0):
        xplat, yplat = transition(state,"left")
        t = (plateau[xplat][yplat],(xplat,yplat))
        listPossibilite.append(t)
        
    if (x+1 <= colonneMax):
        xplat, yplat = transition(state,"right")
        t = (plateau[xplat][yplat],(xplat,yplat))
        listPossibilite.append(t)
        
    if (y-1 >= 0):
        xplat, yplat = transition(state,"up")
        t = (plateau[xplat][yplat],(xplat,yplat))
        listPossibilite.append(t)
        
    if (y+1 < ligneMax):
        xplat, yplat = transition(state,"down")
        t = (plateau[xplat][yplat],(xplat,yplat))
        listPossibilite.append(t)
    choixFinal,posFinal=listPossibilite[0]
    for e, pos in listPossibilite:
        if (e > choixFinal):
            choixFinal=e
            posFinal = pos
    return posFinal

def actualiserPlateau(state):
    y,x = state
    can1.moveto(itemperso,(x*30),(y*30))
    time.sleep(0.5)
    can1.update()
    return 0

#Modele
modele = trouverModele(plateau,500)
print(modele)

#Placement init
oldPosition = start
tabDeplacement=[]
tabDeplacement.append(oldPosition)

#CalculCaseDeplacement
pos=deplacerPerso(oldPosition,modele)
while(oldPosition != pos):
    tabDeplacement.append(pos)
    oldPosition = pos
    pos = deplacerPerso(oldPosition,modele)

#Actualisation du plateau
for sta in tabDeplacement:
    actualiserPlateau(sta)
fen1.mainloop()  

Le modèle à été trouvé en 45 itérations
[[  -6. -104.   -5. -104.   -6. -105. -105.   -6.   -6.   -6.]
 [  -6. -104.   -4. -104.   -6.   -6.   -6.   -6. -105.   -6.]
 [  -5.   -4.   -3.   -4.   -5. -104.   -6.   -6.   -6.   -6.]
 [ -99. -100.   -2. -102. -103. -105.   -6. -105.   -6.   -6.]
 [   1.   -0.   -1.   -2.   -3. -103.   -6.   -6. -105.  -88.]
 [   2.  -98.   -2. -102.   -4.   -5. -104.   -6.  -88.   13.]
 [   3.  -96.   -3.   -4. -104.   -6. -105.  -77.  -80.   16.]
 [   4.    5.  -92.   -5.   -6.  -80.  -77.   26.   22.   19.]
 [ -94.    7.    9.  -88.  -80.   22.   26.   30.  -64.  -59.]
 [ -82.  -90.   11.   13.   16.   19.  -68.   35.   40.   46.]]
